In [669]:
import es_core_news_sm
from gensim.models import Word2Vec, KeyedVectors
import numpy as np
import regex
import pandas as pd
from scipy.spatial.distance import cosine
import os
import nltk

In [670]:
nltk.download('popular')

[nltk_data] Downloading collection 'popular'
[nltk_data]    | 
[nltk_data]    | Downloading package cmudict to
[nltk_data]    |     /home/parker/nltk_data...
[nltk_data]    |   Package cmudict is already up-to-date!
[nltk_data]    | Downloading package gazetteers to
[nltk_data]    |     /home/parker/nltk_data...
[nltk_data]    |   Package gazetteers is already up-to-date!
[nltk_data]    | Downloading package genesis to
[nltk_data]    |     /home/parker/nltk_data...
[nltk_data]    |   Package genesis is already up-to-date!
[nltk_data]    | Downloading package gutenberg to
[nltk_data]    |     /home/parker/nltk_data...
[nltk_data]    |   Package gutenberg is already up-to-date!
[nltk_data]    | Downloading package inaugural to
[nltk_data]    |     /home/parker/nltk_data...
[nltk_data]    |   Package inaugural is already up-to-date!
[nltk_data]    | Downloading package movie_reviews to
[nltk_data]    |     /home/parker/nltk_data...
[nltk_data]    |   Package movie_reviews is already up-to

True

In [790]:
# Funcion para entrenar el modelo
def EntrenarModelo(oraciones,NombreModelo):
    model = Word2Vec(oraciones,vector_size=150, window=20, min_count=1)
    model.save(NombreModelo)
# Funcion para cargar un modelo ya existente
def CargarModelo(NombreModelo):
   modelo = Word2Vec.load(NombreModelo)
   vocabulario = [term for term in modelo.wv.key_to_index]  
   return(modelo,vocabulario)

In [791]:
# Obtener el emebdding de un TEXTO
def ObtenerEmbeddingTexto(modelo, texto):
   Lista_vectores = []
   for w in nltk.word_tokenize(texto, language="spanish"):
       # Verificar que la oracion w exista en el modelo
       try:
           modelo.wv[w]
       except KeyError:
           continue
       # Obtener vector de la oracion
       vec = modelo.wv[w]
       Lista_vectores.append(vec)
   embedding_oraciones = np.array(Lista_vectores)
   if (len(embedding_oraciones) > 0):
        embedding_texto = embedding_oraciones.mean(axis=0)
   else:
        embedding_texto = np.zeros(modelo.vector_size)
   return(embedding_texto) 

In [792]:
# Funcion para crear el corpus
def CrearCorpus(path):
  directorio = os.listdir(path)
  corpus = []
  doc_id = []  
  for filename  in directorio:
     texto = open(path+filename,'r',encoding="UTF-8").read()
     corpus.append(texto)
     doc_id.append(filename)
  return(corpus,doc_id)
# Preprocesamiento de textos
def PreProcesarTextos(textos):
    texto_limpio = []
    # Para cada texto tokenizar por oraciones
    for texto in textos:
        if len(texto) != 0:
            texto_limpio.append(nltk.word_tokenize(texto, language="spanish"))
    return texto_limpio
def PreProcesarTextos2(textos):
    texto_limpio = []
    # Para cada texto tokenizar por oraciones
    for texto in textos:
        if len(texto) != 0:
            texto_limpio.append(nltk.sent_tokenize(texto, language="spanish"))
    return texto_limpio

In [793]:
def CrearDiccionario(lista,claves):
   dicc = {}
   for  v in range(0,len(claves)):
      dicc[claves[v]] = lista[v]
   return(dicc)

In [794]:
PATH = "DiscursosOriginales/"
# Creamos el corpus a partir de todos los textos en el directiorio PATH
corpus,docID = CrearCorpus(PATH)
# Se divide cada texto en oraciones
oraciones = PreProcesarTextos(corpus)
CorpusConClave  = CrearDiccionario(corpus,docID)


In [795]:
# Entrenamos el modelo con los textos tokenizados por oracion y lo llamamos mi_word2vec
EntrenarModelo(oraciones,'mi_word2vec')

In [796]:
# cargamos el modelo y el vocabulario
modelo, vocabulario = CargarModelo('mi_word2vec')

In [797]:
# Ordenar oraciones por puntaje
def centroide(modelo, oraciones):
    embedding = []
    for oracion in oraciones:
        embedding.append(ObtenerEmbeddingTexto(modelo, oracion))
    return np.mean(embedding, axis=0)
def OrdenarOraciones(oraciones):
    puntaje = []
    centroidee = centroide(modelo, oraciones)
    oraciones.sort(key=lambda oracion: cosine(ObtenerEmbeddingTexto(modelo, oracion), centroidee))
    return oraciones

In [798]:
# Metodo summarization imlementado en python
def generar_resumen(O,N,P,U):
    O = OrdenarOraciones(O)
    largo = 1
    Resumen = [""]*(N+1)
    M = len(O)
    for i in range(M):
        if (largo > N): return Resumen[1:N+1]
        Vo = ObtenerEmbeddingTexto(modelo, O[i])
        incluirOracion = False
        for j in range(largo):
            try:
                Vr = ObtenerEmbeddingTexto(modelo,Resumen[j])
            except KeyError:
                Vr = np.zeros(len(Vo))
            Sim = 1-cosine(Vo, Vr)
            if Sim > U and (O[i] not in Resumen):
                incluirOracion = True
        if incluirOracion == True:
            Resumen[largo] = O[i]
            largo+=1
    return Resumen[1:N+1]

In [799]:
# Generamos un resumen de un texto aleatorio
text = open('72186.txt', encoding='UTF-8').read()
oraciones = PreProcesarTextos2([text])[0]
resumen = generar_resumen(oraciones, 10, 0,0.2)
resumen

/home/parker/Documents/env/jupy/lib64/python3.10/site-packages/scipy/spatial/distance.py:620: RuntimeWarning: invalid value encountered in double_scalars
  dist = 1.0 - uv / np.sqrt(uu * vv)


['Y también quisiera no solamente reconocer en este día de alegría y celebración, con verdadero orgullo y con verdadera emoción, a esa generación de héroes y mártires, gracias a cuyas “alas enarcadas en suprema sed de cielo” -como bien dice el himno de la FACH- se ha logrado consolidar la integración de nuestro territorio y la defensa de nuestra soberanía.',
 'Sin embargo, la formación de la Fuerza Aérea de Chile había comenzado mucho antes de que se firmara ese Decreto, gracias al esfuerzo, la creatividad y el coraje de algunos jóvenes aviadores que supieron anticiparse a su tiempo.',
 'Pero lo cierto es que, en la historia de la aviación, Chile siempre ha contado con ese compromiso y entrega que caracterizó a sus jóvenes pioneros y héroes en el pasado.',
 'Yo creo que ese compromiso y ese sentido de misión es lo que ha caracterizado a la Fuerza Aérea de Chile durante sus primeros 88 años de vida, y la va a seguir caracterizando y guiando en los años que vendrán.',
 'De esa manera, só